In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [33]:
class BiRNN(nn.Module):
    def __init__(self, K, MODULUS, hidden_size, num_layers):
        super(BiRNN, self).__init__()
        # Bidirectional RNN layer
        self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, batch_first=True)
        # Fully connected layer for output
        self.fc = nn.Linear(2 * hidden_size, MODULUS)  # 2 * hidden_size because it's bidirectional

    def forward(self, x):
        print(x.shape)
        # Forward pass through RNN
        out, _ = self.rnn(x)
        # Take the output from the last time step
        out_last = out[:, -1, :]
        # Pass it through the fully connected layer
        out_final = self.fc(out_last)
        return out_final

In [39]:
def generate_full_multiple_data(MODULUS: int, K :int):
    ranges = [range(MODULUS)] * K
    combinations = np.array(np.meshgrid(*ranges)).T.reshape(-1, K)
    modulo_results = combinations.sum(axis=1) % MODULUS
    full_data = np.hstack((combinations, modulo_results.reshape(-1, 1)))
    return full_data
full_data = generate_full_multiple_data(10, 4)
full_data = torch.tensor(full_data, dtype=torch.float32)
full_data = full_data.unsqueeze(-1)
full_data[:, :-1,:].shape

torch.Size([10000, 4, 1])

In [40]:

model = BiRNN(K=4, P=10, num_layers=4, hidden_size=5)
model(torch.tensor(full_data[:, :-1,:], dtype=torch.float32)), full_data[:, -1,:].shape

torch.Size([10000, 4, 1])


C:\Users\wangr\AppData\Local\Temp\ipykernel_24348\2239002839.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model(torch.tensor(full_data[:, :-1,:], dtype=torch.float32)), full_data[:, -1,:].shape


(tensor([[ 0.3736,  0.3143,  0.1453,  ..., -0.2166,  0.1038, -0.1546],
         [ 0.3635,  0.3126,  0.1459,  ..., -0.2154,  0.1061, -0.1515],
         [ 0.3597,  0.3207,  0.1388,  ..., -0.2136,  0.1108, -0.1553],
         ...,
         [ 0.4681,  0.3168,  0.0005,  ..., -0.3354,  0.1812, -0.0724],
         [ 0.4861,  0.3215, -0.0085,  ..., -0.3292,  0.1854, -0.0824],
         [ 0.5007,  0.3258, -0.0150,  ..., -0.3244,  0.1882, -0.0909]],
        grad_fn=<AddmmBackward0>),
 torch.Size([10000, 1]))

In [42]:
criteria = nn.CrossEntropyLoss()
criteria(model(torch.tensor(full_data[:, :-1,:], dtype=torch.float32)), full_data[:, -1,:].long().squeeze(-1))

torch.Size([10000, 4, 1])


C:\Users\wangr\AppData\Local\Temp\ipykernel_24348\4182674284.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  criteria(model(torch.tensor(full_data[:, :-1,:], dtype=torch.float32)), full_data[:, -1,:].long().squeeze(-1))


tensor(2.3322, grad_fn=<NllLossBackward0>)